## 7. 位相推定に基づくアルゴリズム

この章では位相推定 (Quantum Phase Estimation) のサブルーサブルーチンを用いた実用的なアルゴリズムを2例紹介する。位相推定はユニタリー演算子の固有値を求める問題に帰着できる問題に適用することができる。
1. 量子多体系ハミルトニアンの固有値問題 (水素分子の規定エネルギー)
2. 逆行列を求める問題 (HHL)

応用例を見る前に、位相推定アルゴリズム

### 7-1. 位相推定再訪
量子位相推定 (Quantum Phase Estimation, QPE) は、適当なユニタリー $U$ が与えられたとき、その固有値 $e^{i \lambda}$ をビット列として取り出すためのアルゴリズムである。

$U$ の固有値 $e^{i \lambda_i}$ に対応する固有ベクトルを $| u_i \rangle$ とする。この時、制御ユニタリ演算 $\Lambda (U)$ と量子フーリエ変換を用いて、次の変換を実現する。
$$
\sum_i \alpha_i | u_i \rangle | 0^{\otimes t} \rangle \xrightarrow{\text{QPE}} \sum_i \alpha_i | u_i \rangle | \tilde{\lambda_i} \rangle
$$
ただし $\tilde{\lambda_i}$ は $\lambda_i$ を2進展開したビット列:
$$
\frac{\tilde{\lambda_i}}{2 \pi} = \frac{j_1}{2} + \frac{j_2}{2^2} + \ldots + \frac{j_t}{2^t} = 0.j_1 j_2 \ldots j_t
$$ 
である (詳細は[2-4. 位相推定アルゴリズム（入門編）](https://github.com/qulacs/quantum-native-dojo/blob/master/notebooks/2.4_phase_estimation_beginner.ipynb) を参照)。

この位相推定アルゴリズムをサブルーチンとして用いると、素因数分解や量子多体系のエネルギー計算といった固有値問題に帰着できる多くの問題を、古典コンピュータと比べて効率よく (指数的に高速) 解けることが期待されている事にも触れた。

ここで注意が必要なのは、このアルゴリズムは任意の制御ユニタリーゲート $\Lambda (U^{2k})$ ($0 \leq k \leq t$) が用意できると仮定している事である。実際、後で述べるように制御ユニタリーゲートを実機で実行可能なゲートセットに分解して構成したり、固有値のビット列を取り出す際の逆フーリエ変換を実行するには、一般的に量子回路が深くなってしまい長時間コヒーレント状態を維持する必要がある。そのため誤り訂正機能を備えていないNISQ デバイスでは実現不可能と考えられている。

最後の逆フーリエ変換で用いられている制御位相ゲート $\Lambda (R_l)$ と、制御ビットの測定と測定結果 $m_k$ を用いて位相ゲートをかける操作 ($R^{m}_l$) は可換である事を用いて、必要な補助ビットを削減する方法も存在する。

反復的位相推定法 (Iterative Quantum Phase Estimation, IQPE) は固有値を2進展開したビット列を、各桁ごとに決定的に求める方法である。必要な補助ビットは 1つで、1イテレーションごとに1つの桁の値 ($j_k$) を求める。
手順は以下の通りである ($k = t, t-1, \ldots, 1$):

#### $k = t$ の時
1. 補助ビットに $H$ ゲートをかける
$$
| u_i \rangle | 0 \rangle \xrightarrow{H} | u_i \rangle | + \rangle 
$$
2. 補助ビットに $\Lambda (U^{2(t-1)})$ をかける
$$
| u_i \rangle | + \rangle \xrightarrow{\Lambda (U^{2t})} | u_i \rangle (| 0 \rangle + e^{-i \pi j_t} | 1 \rangle)
$$
3. 補助ビットに $H$ ゲートをかけて測定する
$$
| u_i \rangle (| 0 \rangle + e^{-i \pi j_t} | 1 \rangle) \xrightarrow{H} | u_i \rangle [(1 + e^{-i \pi j_t})| 0 \rangle + (1 - e^{-i \pi j_t}) | 1 \rangle)] \xrightarrow{\textrm{Measure}} | u_i \rangle | j_t \rangle
$$

#### $k \leq t-1$ の時
1. 補助ビットに $H$ ゲートをかける
$$
| u_i \rangle | 0 \rangle \xrightarrow{H} | u_i \rangle | + \rangle 
$$
2. 補助ビットに $R_Z (\pi 0.j_{k+1} j_{k+2} \ldots j_{t})$ をかける
$$
| u_i \rangle | + \rangle \xrightarrow{H} | u_i \rangle (| 0 \rangle + e^{+i \pi 0.j_{k+1} j_{k+2} \ldots j_{t}} | 1 \rangle)
$$
3. 補助ビットに $\Lambda (U^{2(k-1)})$ をかける
$$
| u_i \rangle (| 0 \rangle + e^{+i \pi 0.j_{k+1} j_{k+2} \ldots j_{t}} | 1 \rangle) \xrightarrow{\Lambda (U^{2(k-1)})} | u_i \rangle (| 0 \rangle + e^{-i \pi j_k} | 1 \rangle)
$$
4. 補助ビットに $H$ ゲートをかけて測定する
$$
| u_i \rangle (| 0 \rangle + e^{-i \pi j_k} | 1 \rangle) \xrightarrow{H} | u_i \rangle [(1 + e^{-i \pi j_k})| 0 \rangle + (1 - e^{-i \pi j_k}) | 1 \rangle)] \xrightarrow{\textrm{Measure}} | u_i \rangle | j_k \rangle
$$




### 3. 実践例： 位相推定を使った分子の基底状態エネルギーの計算

この章では分子の基底状態を求める例を実際に量子回路を構成して行う。
水素分子を例に
位相推定ではハミルトニアンを精度よく近似する事が精度よくエネルギーを測定する上で重要であり、
そのためには高次の Trotter-Suzuki 展開を行う必要がある。
しかし、後で見るように任意の controlled-$U^{2k}$ を準備するため、逆フーリエ変換するための回路が深くなり、
誤り訂正可能な FTQC 上で実行する必要があるため、Long-term に分類されている。

任意の controlled unitary operator が準備できると仮定して議論されていた。$C(U2k)$
固有状態が用意できると仮定 $ket u$
実際は
elementary gates に分解して実装 (polynomial 個数のゲート) する必要がある。Trotter-Suzuki 
ゲート数削減のための工夫 (symmetry の利用, commuting terms を落とす)
$C(R_Z(t))$, $C(R_{XX}())$, $C(R_{YY}())$ の実装
exact な固有状態は用意できないが十分 overlap のある状態は作れる。$psi = sum c_n ket n$
binary pre$$



Cannonical な方法
$exp[-i H]$ の固有値が求められればハミルトニアン $H$ の固有エネルギー値が求まる。 

Scalable Quantum Simulation of Molecular Energies,
P. J. J. O’Malley et. al.,
PHYSICAL REVIEW X 6, 031007 (2016)

Hartree-Fock (HF) 状態 $|\phi \rangle$ に時間発展演算子 $e^{−iHt}$ を作用させる。
$|n \rangle$ をハミルトニアンの固有状態、それに対応する固有値を $E_n$ とすると、

時間発展演算子
$$
e^{−iHt} |\phi \rangle = \sum_n e^{−i E_n t} \langle n |\phi \rangle |n \rangle 
$$

この状態を補助ビットとエンタングルさせ、補助ビットの $|0 \rangle$ と $|1 \rangle$ 間の位相を測定する。
$$
e^{−iHt} |+^m \rangle |\phi \rangle \to \sum_n e^{−i E_n t} \langle n |\phi \rangle |E_n \rangle |n \rangle 
$$
測定が行われると、状態が $|\langle n |\phi \rangle|^2$ の確率で $|n \rangle $ に collaps する。
ことで、ハミルトニアンの固有値を求める。


量子コンピュータデバイス上で時間発展演算子を準備するためには、量子ゲートに分解する必要がある。
上式を Trotter-Suzuki 展開を用いて $exp(i \theta P)$ の積の形に近似する必要がある。
ただしここで $P$ はパウリ行列の積である。

The time-evolution operator can be approximated using Trotterization [34] as

$$
e^{−iHt} = exp[−it \sum_i g_i H_i] \approx U_{Torrot} (t) = \left( \prod_i \exp[-i g_i H_i t/n] \right)^n
$$
この近似ではエラーが $n$ に線形で依存することが知られている。

### 水素分子の基底状態エネルギー計算
水素分子の第二量子化されたハミルトニアン (STO-6G 基底) を Bravyi-Kitaev 変換した後、電子数保存などの対称性を用いて系の次元を削減すると、電子状態は 2量子ビットで表される (詳しくは文献 [1] を参照されたい)。

$$
H = \sum_i g_i H_i = g_0 I + g_1 Z_0 + g_2 Z_1 + g_3 Z_0 Z_1 + g_4 Y_0 Y_1 + g_5 X_0 X_1
$$
係数 $g_i$ は実数で値は原子間距離に依存する。

TODO: trotter-suzuki order and error

IQPE
Our PEA implementation is based on a modification of Kitaev’s iterative phase estimation algorithm [8,35]. The circuit 

$U_{Torrot} (2^k t)$

補助ビット (ancilla bit) を $|+>$ の状態に準備する。

### 参考文献
[1] Scalable Quantum Simulation of Molecular Energies,
P. J. J. O’Malley et. al.,
PHYSICAL REVIEW X 6, 031007 (2016) <br>

### 1. Quantum random access memory (qRAM) とは
量子コンピュータ上で古典データを扱う際は、そのデータを量子状態としてエンコードする必要がある。とくにバイナリデータの集まり（ベクトル）を量子状態として効率的に読み出すことは、量子機械学習などの応用において極めて重要である。本節ではqRAM (quantum random acccess memory)について解説する。

古典コンピュータにおけるRAM (random access memory)とは、メモリアドレスと対応するデータをセットとして格納し、引き出せるようにする装置である。すなわち、RAMにメモリアドレス$i$を与えると、バイナリデータ$x_i$ を引き出すことができる。

$$ i \quad \to \quad x_i $$

同様に、qRAMはメモリアドレスに対応する量子状態 $|i \rangle$ から、対応するデータをエンコードした量子状態 $|x_i \rangle$（以下、データ状態とよぶ）を引き出せるようにする装置である。

$$|i \rangle \quad \to \quad |i \rangle \otimes |x_i \rangle$$

とくに、メモリアドレスとデータを重ね合わせ状態として引き出せるということは、qRAMのもつべき重要な性質である。すなわちメモリアドレス状態の重ね合わせに対して、qRAMはアドレスとデータのエンタングル状態を与える。
$$ \sum_{i=1}^N  |i \rangle \quad \to \quad \sum_{i=1}^N |i \rangle \otimes |x_i \rangle $$

ここで$N$はデータの件数である。

qRAMは必ずしも量子状態そのものを保持する必要がないということに注意されたい。すなわちバイナリデータのベクトルが与えられたとき、効率的に重ね合わせ状態を生成する量子回路の記述が与えられれば、qRAMとしての役割を果たす。qRAMの仕組みを実現するアーキテクチャにはとくに決まったものはなく、現在も研究途上である。

`Quantum Random Access Memory (QRAM)` とは、あるバイナリデータ $x_i$ のアドレス $i$ を記述する量子ビット列 $| i \rangle $ が与えられたとき、そのデータ $x_i$ を量子ビット列 $| x_i \rangle$ として取り出す機能のことである。
具体的には、次のような機能が QRAM と呼ばれることが多いようだ。
$$ | i \rangle | 0 \rangle \xrightarrow{QRAM} | i \rangle | x_i \rangle $$
通常 QRAM はユニタリーなプロセスとして実現することが仮定される。したがって、QRAMの存在を仮定するとき、以下のように重ね合わせ状態を入力すれば、同時並列的にデータを量子状態にエンコードすることが可能である。
$$ \frac{1}{\sqrt{N}}\sum_{i=1}^N | i \rangle | 0 \rangle \xrightarrow{QRAM} \frac{1}{\sqrt{N}}\sum_i | i \rangle | x_i \rangle $$
現実にこれが可能であるかどうかはまだわからない。具体的な実装方法としては 
V. Giovannetti, S. Lloyd, L. Maccone, Phys. Rev. A 78, 052310 (2008) 
10.1103/PhysRevA.78.052310
などで提案がある。


## 振幅エンコーディング
HHLアルゴリズムおよびそれをベースとする機械学習アルゴリズムでは、qRAM上のデータを状態としてではなく、振幅として利用したい場合がある。そのためには、qRAMから読み出したデータに対して次のような変換を行いたい。

$$ \sum_{i=1}^N |i \rangle \otimes |x_i \rangle \quad \to \quad \frac{1}{||{\\bf x}||} \sum_{i=1}^N x_i |i \rangle $$

ここで $||{\\bf x}||$ はベクトル $x_i$ のノルムである。この変換をユニタリ変換として実現する方法はPrakashの博士論文において提案された[1]。具体的な実現方法の解説は[2,3]に詳しい。

### 参考文献
[1] Anupam Prakash. \"Quantum Algorithms for Linear Algebra and Machine Learning\". PhD thesis, EECS Department, University of California, Berkeley, Dec 2014. <br>
[2] https://www2.yukawa.kyoto-u.ac.jp/~qischool2019/mitaraiCTO.pdf\n", <br>
[3] Danial Dervovic, et al. \"Quantum linear systems algorithms: a primer\". eprint: https://arxiv.org/abs/1802.08227"

### 4. HHLアルゴリズム (コードはある 

### コラム：低ランクに対するquantum inspired algorithm <- 鈴木さんの記事がある

### 5. 実践例：HHLを使ったポートフォリオ最適化 (すでにある 